# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [ ]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import the model and preferences
import inauguralproject
from inauguralproject import HouseholdSpecializationModelClass as HSMC


# Question 1

**Explain how you solve the model**

In [ ]:
# create instance of the model
model1 = HSMC()

# define values of alpha and sigma to loop through
alpha_values = [0.25, 0.5, 0.75]
sigma_values = [0.5, 1.0, 1.5]

# initialize dictionary to store results
resultsHF = {}
resultsHM = {}

# loop through all combinations of alpha and sigma
for alpha in alpha_values:
    for sigma in sigma_values:
        # set values of alpha and sigma in the model
        model1.par.alpha = alpha
        model1.par.sigma = sigma
        
        # solve the model and store the result
        opt = model1.solve_discrete()
        resultsHF[(alpha, sigma)] = opt.HF
        resultsHM[(alpha, sigma)] = opt.HM

# plot the results
fig, ax = plt.subplots()
for alpha in alpha_values:
    y = [resultsHF[(alpha, sigma)]/resultsHM[(alpha,sigma)] for sigma in sigma_values]
    ax.plot(sigma_values, y, label=f"alpha={alpha}")
ax.set_xlabel("sigma")
ax.set_ylabel("HF/HM")
ax.legend()
plt.show()


# Question 2

Explain your code and procedure

In [ ]:
# create instance of the model
model2 = HSMC()

# define values of alpha and sigma to loop through
wF_values = [0.8, 0.9, 1.0, 1.1, 1.2]


# initialize dictionary to store results
resultsHF = {}
resultsHM = {}

for wF in wF_values:
    # set values of alpha and sigma in the model
    model2.par.wF = wF
        
    # solve the model and store the result
    opt = model2.solve_discrete()
    resultsHF[wF] = opt.HF
    resultsHM[wF] = opt.HM

# plot the results
fig, ax = plt.subplots()
for wf in wF_values:
    y = [np.log(resultsHF[wF]/resultsHM[wF]) for wF in wF_values]
ax.plot(np.log(wF_values),y)
ax.set_xlabel("wF")
ax.set_ylabel("HF/HM")
plt.show()

print(resultsHF)
print(resultsHM)



Når den relative løn for kvinder falder, vil de bruge mere tid hjemme... noget mere :)

# Question 3

Explain your code and procedure

In [ ]:
model3 = HSMC()

# define values of alpha and sigma to loop through
wf_values = [0.8, 0.9, 1.0, 1.1, 1.2]


# initialize dictionary to store results
resultsHF = {}
resultsHM = {}

for i,wF in enumerate(wf_values):
    # set values of alpha and sigma in the model
    model3.par.wF = wF
        
    # solve the model and store the result
    opt = model3.solve()
    resultsHF[wF] = opt.HF
    resultsHM[wF] = opt.HM

    model3.sol.HF_vec[i] = opt.HF
    model3.sol.HM_vec[i] = opt.HM
    model3.sol.LF_vec[i] = opt.LF
    model3.sol.LM_vec[i] = opt.LM



# plot the results
fig, ax = plt.subplots()
for wf in wF_values:
    y = [np.log(resultsHF[wF]/resultsHM[wF]) for wF in wF_values]
ax.plot(np.log(wF_values),y)
ax.set_xlabel("log(wF)")
ax.set_ylabel("log(HF/HM)")
plt.show()

print(resultsHF)
print(resultsHM)

## Question 4

In [ ]:
# Printing the result of our regression

print(model3.run_regression())


In [ ]:
# Defining our estimate to minimize the squared differences
def estimate(self, alpha=None, sigma=None):
    par = self.par
    sol = self.sol

    diff0 = par.beta0_target - model3.run_regression(sol.beta0)
    diff1 = par.beta1_target - model3.run_regression(sol.beta1)
   
    return diff0**2 + diff1**2

# Providing initial guess
x0 = [0.5, 1]

# Minimizing the squared differences with our optimize/minimize
res = optimize.minimize(estimate, x0)

print(res)